# Imports & Configuration

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
class Config:
    # FILE PATH: Change this if your file is in a different folder
    FILE_PATH = "data/aap9112_data_file_s1.xlsx" 
    
    # Model Hyperparameters (Optimized)
    EMBEDDING_DIM = 64
    HIDDEN_DIM = 128
    NUM_LAYERS = 2
    DROPOUT_RATE = 0.2
    BATCH_SIZE = 32
    LEARNING_RATE = 0.001
    EPOCHS = 40
    
    # Device (GPU if available, else CPU)
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"✅ Setup Complete. Using Device: {Config.DEVICE}")

✅ Setup Complete. Using Device: cpu


# Chemical Mappings (The Dictionary)

In [2]:
# --- 1. Reactant 1 (Quinoline Derivatives) ---
r1_map = {
    '1a, 6-Cl-Q': 'Clc1ccc2ncccc2c1',
    '1b, 6-Br-Q': 'Brc1ccc2ncccc2c1',
    '1c, 6-OTf-Q': 'O=S(=O)(Oc1ccc2ncccc2c1)C(F)(F)F',
    '1d, 6-I-Q': 'Ic1ccc2ncccc2c1',
    '1e, 6-BOH2-Q': 'OB(O)c1ccc2ncccc2c1',
    '1f, 6-BPin-Q': 'CC1(C)OB(Oc1ccc2ncccc2c1)OC1(C)C',
    '1g, 6-BF3K-Q': 'F[B-](F)(F)c1ccc2ncccc2c1.[K+]'
}

# --- 2. Reactant 2 (Pyridine Partners) ---
r2_map = {
    '2a, Boronic Acid': 'OB(O)c1cnccc1',
    '2b, Boronic Ester': 'CC1(C)OB(Oc1cnccc1)OC1(C)C',
    '2c, Trifluoroborate': 'F[B-](F)(F)c1cnccc1.[K+]',
    '2d, Bromide': 'Brc1cnccc1'
}

# --- 3. Ligands ---
lig_map = {
    'P(tBu)3': 'CC(C)(C)P(C(C)(C)C)C(C)(C)C',
    'P(Ph)3 ': 'c1ccccc1P(c2ccccc2)c3ccccc3',
    'AmPhos': 'CN(C)c1ccc(P(C(C)(C)C)C(C)(C)C)cc1',
    'P(Cy)3': 'C1CCCCC1P(C2CCCCC2)C3CCCCC3',
    'P(o-Tol)3': 'Cc1ccccc1P(c2ccccc2C)c3ccccc3C',
    'CataCXium A': 'CCCCP(C12CC3CC(C1)CC(C3)C2)C45CC6CC(C4)CC(C6)C5',
    'SPhos': 'COc1cccc(OC)c1-c2ccccc2P(C3CCCCC3)C4CCCCC4',
    'dtbpf': 'CC(C)(C)P(C1=C[C-]=C1)C(C)(C)C.CC(C)(C)P(C1=C[C-]=C1)C(C)(C)C.[Fe+2]',
    'XPhos': 'CC(C)c1cc(C(C)C)cc(C(C)C)c1-c2ccccc2P(C3CCCCC3)C4CCCCC4',
    'dppf': 'c1ccc(cc1)P(c2ccccc2)C3=C[C-]=C3.c1ccc(cc1)P(c2ccccc2)C3=C[C-]=C3.[Fe+2]',
    'Xantphos': 'CC1(C)c2cccc(P(c3ccccc3)c4ccccc4)c2Oc5c1cccc5P(c6ccccc6)c7ccccc7',
    'None': ''
}

# --- 4. Reagents & Solvents ---
reag_map = {
    'NaOH': '[Na+].[OH-]', 'NaHCO3': '[Na+].OC(=O)[O-]', 'CsF': '[Cs+].[F-]',
    'K3PO4': '[K+].[K+].[K+].[O-]P(=O)([O-])[O-]', 'KOH': '[K+].[OH-]',
    'LiOtBu': '[Li+].[O-]C(C)(C)C', 'Et3N': 'CCN(CC)CC', 'None': ''
}

solv_map = {
    'MeCN': 'CC#N', 'THF': 'C1CCOC1', 'DMF': 'CN(C)C=O', 'MeOH': 'CO',
    'MeOH/H2O_V2 9:1': 'CO.O', 'THF_V2': 'C1CCOC1'
}

cat_map = {'Pd(OAc)2': 'CC(=O)O[Pd]OC(C)=O'}

print("✅ Chemical Mappings Loaded.")

✅ Chemical Mappings Loaded.


# Helpers & Model Definition

In [3]:
# --- 1. Tokenizer (Regex) ---
def smiles_tokenizer(smiles):
    pattern =  r"(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
    regex = re.compile(pattern)
    tokens = [token for token in regex.findall(smiles)]
    return tokens

# --- 2. Padding Function ---
def pad_sequence(seq, max_len):
    return seq + [0] * (max_len - len(seq))

# --- 3. Data Processing Pipeline ---
def prepare_data(filepath):
    print("📂 Loading Data...")
    df = pd.read_excel(filepath, engine='openpyxl')
    
    # Map names to SMILES
    df['Reaction_SMILES'] = (
        df['Reactant_1_Short_Hand'].map(r1_map) + "." +
        df['Reactant_2_Name'].map(r2_map) + "." +
        df['Catalyst_1_Short_Hand'].map(cat_map) + "." +
        df['Ligand_Short_Hand'].map(lig_map) + "." +
        df['Reagent_1_Short_Hand'].map(reag_map) + "." +
        df['Solvent_1_Short_Hand'].map(solv_map)
    )
    
    # Remove bad rows
    df = df.dropna(subset=['Reaction_SMILES'])
    print(f"   Rows loaded: {len(df)}")
    
    # Build Vocabulary
    print("   Building Vocabulary...")
    all_tokens = [smiles_tokenizer(s) for s in df['Reaction_SMILES'].tolist()]
    vocab = sorted(list(set([item for sublist in all_tokens for item in sublist])))
    vocab_to_int = {c: i+1 for i, c in enumerate(vocab)}
    max_length = max(len(seq) for seq in all_tokens)
    
    print(f"   Vocab Size: {len(vocab)}, Max Length: {max_length}")
    
    # Text -> Numbers
    sequences = []
    for tokens in all_tokens:
        encoded = [vocab_to_int[t] for t in tokens]
        sequences.append(pad_sequence(encoded, max_length))
        
    X = np.array(sequences)
    y = df['Product_Yield_PCT_Area_UV'].values / 100.0 # Normalize 0-1
    
    return X, y, vocab_to_int, max_length, len(vocab) + 1

# --- 4. The Neural Network (GRU) ---
class ReactionYieldPredictor(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate):
        super(ReactionYieldPredictor, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(
            input_size=embedding_dim, 
            hidden_size=hidden_dim, 
            num_layers=num_layers, 
            batch_first=True, 
            dropout=dropout_rate
        )
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        output, ht = self.gru(embedded)
        # Take the last hidden state of the top layer
        return self.sigmoid(self.fc(ht[-1]))

print("✅ Model Architecture & Helpers Defined.")

✅ Model Architecture & Helpers Defined.


# Training & Saving

In [4]:
# 1. Prepare Data
X, y, vocab_to_int, max_length, vocab_size = prepare_data(Config.FILE_PATH)

# 2. Split Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Create DataLoaders
train_data = TensorDataset(torch.LongTensor(X_train), torch.FloatTensor(y_train).view(-1, 1))
test_data = TensorDataset(torch.LongTensor(X_test), torch.FloatTensor(y_test).view(-1, 1))

train_loader = DataLoader(train_data, batch_size=Config.BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=Config.BATCH_SIZE)

# 4. Initialize Model
model = ReactionYieldPredictor(
    vocab_size, Config.EMBEDDING_DIM, Config.HIDDEN_DIM, Config.NUM_LAYERS, Config.DROPOUT_RATE
).to(Config.DEVICE)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=Config.LEARNING_RATE)

# 5. Train Loop
print(f"\n🚀 Starting Training ({Config.EPOCHS} Epochs)...")
for epoch in range(Config.EPOCHS):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{Config.EPOCHS}", leave=False)
    
    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(Config.DEVICE), targets.to(Config.DEVICE)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
        
    if (epoch + 1) % 5 == 0:
        print(f"   Epoch {epoch+1} | Avg Loss: {total_loss/len(train_loader):.4f}")

# 6. Evaluation
model.eval()
predictions, actuals = [], []
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(Config.DEVICE)
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(targets.numpy())

r2 = r2_score(actuals, predictions)
mae = mean_absolute_error(actuals, predictions) * 100
print(f"\n📊 Final Results:\n   R² Score: {r2:.3f}\n   Mean Absolute Error: {mae:.2f}%")

# 7. Save Model & Artifacts
torch.save(model.state_dict(), "reaction_yield_model.pth")

artifacts = {
    'vocab_to_int': vocab_to_int,
    'max_length': max_length,
    'vocab_size': vocab_size,
    'config': {
        'embedding_dim': Config.EMBEDDING_DIM,
        'hidden_dim': Config.HIDDEN_DIM,
        'layers': Config.NUM_LAYERS,
        'dropout': Config.DROPOUT_RATE
    }
}
with open("model_artifacts.pkl", "wb") as f:
    pickle.dump(artifacts, f)

print("\n💾 Model Saved as 'reaction_yield_model.pth' & 'model_artifacts.pkl'")

📂 Loading Data...
   Rows loaded: 4620
   Building Vocabulary...
   Vocab Size: 36, Max Length: 166

🚀 Starting Training (40 Epochs)...


   Epoch 5 | Avg Loss: 0.0381


   Epoch 10 | Avg Loss: 0.0252


   Epoch 15 | Avg Loss: 0.0185


   Epoch 20 | Avg Loss: 0.0141


   Epoch 25 | Avg Loss: 0.0112


   Epoch 30 | Avg Loss: 0.0092


   Epoch 35 | Avg Loss: 0.0074


   Epoch 40 | Avg Loss: 0.0061

📊 Final Results:
   R² Score: 0.845
   Mean Absolute Error: 7.67%

💾 Model Saved as 'reaction_yield_model.pth' & 'model_artifacts.pkl'


# The "Optimization" Tool

In [5]:
def optimize_reaction_detailed(r1_name, r2_name):
    """
    Tests all conditions for specific reactants and returns a detailed 'Recipe Card'.
    """
    if r1_name not in r1_map or r2_name not in r2_map:
        print(f"❌ Error: Could not find reactants '{r1_name}' or '{r2_name}'")
        return None
        
    r1_smi = r1_map[r1_name]
    r2_smi = r2_map[r2_name]
    catalyst_name = 'Pd(OAc)2'
    catalyst_smi = cat_map[catalyst_name]
    
    print(f"🔬 Optimizing Reaction: {r1_name} + {r2_name} ...")

    experiments = []
    
    # Loop through all combinations
    for lig_name, lig_smi in lig_map.items():
        if lig_name == 'None': continue
        for solv_name, solv_smi in solv_map.items():
            for reag_name, reag_smi in reag_map.items():
                if reag_name == 'None': continue
                
                # Construct SMILES
                full_smiles = f"{r1_smi}.{r2_smi}.{catalyst_smi}.{lig_smi}.{reag_smi}.{solv_smi}"
                
                experiments.append({
                    'Reactant_1': r1_name,
                    'Reactant_2': r2_name,
                    'Catalyst': catalyst_name,
                    'Ligand': lig_name, 
                    'Solvent': solv_name, 
                    'Reagent': reag_name, 
                    'SMILES': full_smiles
                })
    
    # Process Data
    sequences = []
    for exp in experiments:
        tokens = smiles_tokenizer(exp['SMILES'])
        seq = [vocab_to_int.get(t, 0) for t in tokens]
        seq = pad_sequence(seq, max_length)
        sequences.append(seq)
        
    input_tensor = torch.LongTensor(sequences).to(Config.DEVICE)
    
    # Predict
    model.eval()
    with torch.no_grad():
        predictions = model(input_tensor).cpu().numpy().flatten() * 100
        
    results_df = pd.DataFrame(experiments)
    results_df['Predicted_Yield (%)'] = predictions
    
    # Clean up output table
    cols = ['Reactant_1', 'Reactant_2', 'Catalyst', 'Ligand', 'Solvent', 'Reagent', 'Predicted_Yield (%)']
    results_df = results_df[cols]
    
    top_results = results_df.sort_values(by='Predicted_Yield (%)', ascending=False).head(10).reset_index(drop=True)
    return top_results

# --- EXAMPLE RUN ---
r1_input = '1a, 6-Cl-Q'       # Change this to test other reactants
r2_input = '2a, Boronic Acid' # Change this to test other reactants

top_conditions = optimize_reaction_detailed(r1_input, r2_input)

# Display Table with Colors
if top_conditions is not None:
    cm = sns.light_palette("green", as_cmap=True)
    display(top_conditions.style.background_gradient(cmap=cm, subset=['Predicted_Yield (%)']))

🔬 Optimizing Reaction: 1a, 6-Cl-Q + 2a, Boronic Acid ...


,Reactant_1,Reactant_2,Catalyst,Ligand,Solvent,Reagent,Predicted_Yield (%)
0,"1a, 6-Cl-Q","2a, Boronic Acid",Pd(OAc)2,XPhos,MeCN,NaOH,93.440910
1,"1a, 6-Cl-Q","2a, Boronic Acid",Pd(OAc)2,XPhos,MeCN,LiOtBu,93.324417
2,"1a, 6-Cl-Q","2a, Boronic Acid",Pd(OAc)2,SPhos,MeCN,NaOH,92.809937
3,"1a, 6-Cl-Q","2a, Boronic Acid",Pd(OAc)2,SPhos,MeCN,LiOtBu,92.808075
4,"1a, 6-Cl-Q","2a, Boronic Acid",Pd(OAc)2,XPhos,THF_V2,NaOH,91.760269
5,"1a, 6-Cl-Q","2a, Boronic Acid",Pd(OAc)2,XPhos,THF,NaOH,91.760269
6,"1a, 6-Cl-Q","2a, Boronic Acid",Pd(OAc)2,SPhos,THF,NaOH,90.025696
7,"1a, 6-Cl-Q","2a, Boronic Acid",Pd(OAc)2,SPhos,THF_V2,NaOH,90.025696
8,"1a, 6-Cl-Q","2a, Boronic Acid",Pd(OAc)2,XPhos,MeCN,NaHCO3,88.945679
9,"1a, 6-Cl-Q","2a, Boronic Acid",Pd(OAc)2,SPhos,DMF,NaOH,88.937363
